# Redes neuronales hibridas  para clasificación multiple


In [ ]:
# Install the relevant packages.
#%pip install --upgrade pip
#%pip install torch torchvision torchaudio
#%pip install cudaq -> ya viene instalado en el braket de AWS

In [ ]:
# Check installed clasico
import sys
import numpy as np
import matplotlib
print(f"Python version: {sys.version}")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {matplotlib.__version__}")

import torch, torchvision, torchaudio
print("torch:", torch.__version__)
print("vision:", torchvision.__version__)
print("audio:", torchaudio.__version__)

Python version: 3.12.12 (main, Oct 10 2025, 00:00:00) [GCC 15.2.1 20250808 (Red Hat 15.2.1-1)]
PyTorch version: 2.9.0+cpu
NumPy version: 2.2.6
Matplotlib version: 3.10.7
CUDA available: False


In [ ]:

import cudaq
print(f"CUDAQ version: {cudaq.__version__}")
print(f"Running on target: {cudaq.get_target().name}")

In [ ]:
import sys
import cudaq

print(f"Running on target {cudaq.get_target().name}")
qubit_count = int(sys.argv[1]) if 1 < len(sys.argv) else 2


@cudaq.kernel
def kernel():
    qubits = cudaq.qvector(qubit_count)
    h(qubits[0])
    for i in range(1, qubit_count):
        x.ctrl(qubits[0], qubits[i])
    mz(qubits)


result = cudaq.sample(kernel)
print(result)  # Example: { 11:500 00:500 }

In [ ]:
# importar las librerías necesarias
import cudaq
from cudaq import spin

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Para asegurar que los resultados sean reproducibles
torch.manual_seed(42)

cudaq.set_random_seed(44)

In [ ]:
# Configurar el dispositivo
# Set CUDAQ and PyTorch to run on either CPU or GPU.

device = torch.device('cpu')
cudaq.set_target("qpp-cpu")

#cudaq.set_target("nvidia")
#device = torch.device("cuda:0")

## Descripción del Conjunto de Datos MNIST

- *¿Qué es?:* MNIST (Modified National Institute of Standards and Technology database) es una gran base de datos de dígitos escritos a mano, del 0 al 9.
- *Contenido:* Contiene 70,000 imágenes en escala de grises.
- *Conjunto de entrenamiento:* 60,000 imágenes.
- *Conjunto de prueba:* 10,000 imágenes.
- *Formato de imagen:* Cada imagen tiene un tamaño de 28x28 píxeles.
- *Uso común:* Es considerado el "Hola, Mundo" de la visión por computadora y el aprendizaje profundo. Se utiliza para entrenar y probar algoritmos de clasificación de imágenes.


Paso 2: Cargar, Transformar y Previsualizar los Datos
torchvision nos facilita la descarga y preparación de datasets.

Transformaciones: Convertimos las imágenes a tensores de PyTorch y las normalizamos. La normalización (ajustar los valores de los píxeles para que tengan una media de 0.5 y una desviación estándar de 0.5) ayuda a que el modelo entrene más rápido y de forma más estable.
Descarga: Descargamos los conjuntos de entrenamiento y prueba. FashionMNIST ya viene separado en train y test.

In [ ]:
# 1. Definir las transformaciones para las imágenes
# - transforms.ToTensor() convierte la imagen (PIL) a un Tensor de PyTorch.
# - transforms.Normalize() ajusta los valores del tensor para que tengan una media y desviación estándar específicas.
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)) # Media y Desviación Estándar para un solo canal (escala de grises)
])

# 2. Descargar los datasets de entrenamiento y prueba
train_full_dataset = torchvision.datasets.MNIST(
    root='./data', 
    train=True, 
    download=True, 
    transform=transform
)

test_dataset = torchvision.datasets.MNIST(
    root='./data', 
    train=False, 
    download=True, 
    transform=transform
)

print(f"Tamaño total del dataset de entrenamiento: {len(train_full_dataset)}")
print(f"Tamaño del dataset de prueba: {len(test_dataset)}")


In [ ]:
# Definimos los tamaños para la división train/validation
train_size = int(0.8 * len(train_full_dataset)) # 80% para entrenamiento
val_size = len(train_full_dataset) - train_size # 20% para validación

# Dividimos el dataset de entrenamiento
train_dataset, val_dataset = random_split(train_full_dataset, [train_size, val_size])

print(f"Tamaño del subconjunto de entrenamiento: {len(train_dataset)}")
print(f"Tamaño del subconjunto de validación: {len(val_dataset)}")

# Crear los DataLoaders
# Los DataLoaders nos permiten iterar sobre los datos en lotes (batches)
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# Clases de MNIST (dígitos del 0 al 9)
classes = tuple(str(i) for i in range(10))

# Función para mostrar imágenes (sin cambios)
def imshow(img):
    img = img / 2 + 0.5  # Des-normalizar
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Obtener un lote de imágenes de entrenamiento
dataiter = iter(train_loader)
images, labels = next(dataiter)

# Mostrar las primeras 8 imágenes y sus etiquetas
print("--- Previsualización de Datos (MNIST) ---")
imshow(torchvision.utils.make_grid(images[:8]))
print('Etiquetas: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(8)))


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            # Capa 1: Entrada (784 neuronas) -> Oculta 1 (512 neuronas)
            nn.Linear(28*28, 512),
            nn.ReLU(),
            # Capa 2: Oculta 1 (512) -> Oculta 2 (256 neuronas)
            nn.Linear(512, 256),
            nn.ReLU(),
            # Capa 3: Oculta 2 (256) -> Oculta 3 (128 neuronas)
            nn.Linear(256, 128),
            nn.ReLU(),
            # Capa 4 (Salida): Oculta 3 (128) -> Salida (10 neuronas)
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# Instanciamos el modelo y lo movemos al dispositivo (GPU/CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = NeuralNetwork().to(device)
print("\n--- Arquitectura del Modelo ---")
print(model)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"Pérdida (loss): {loss:>7f}  [{current:>5d}/{size:>5d}]")

def validation_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    val_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= num_batches
    correct /= size
    print(f"Resultado: \n Precisión (Accuracy): {(100*correct):>0.1f}%, Pérdida promedio: {val_loss:>8f} \n")

# Bucle principal de entrenamiento
epochs = 10
print("\n--- Iniciando Entrenamiento con MNIST ---")
for t in range(epochs):
    print(f"Época {t+1}\n-------------------------------")
    print("Entrenando...")
    train_loop(train_loader, model, loss_fn, optimizer)
    print("Validando...")
    validation_loop(val_loader, model, loss_fn)
print("¡Entrenamiento finalizado!")


In [ ]:
print("\n--- Evaluando con el conjunto de Prueba (Test) ---")
validation_loop(test_loader, model, loss_fn)
